In [8]:
import pandas as pd
# applying progress bar
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv('../data/raw/train.csv')
print('Total rows', len(df))

C:\Users\tommy\anaconda3\envs\py37\lib\site-packages\tqdm\std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Total rows 955454


In [2]:
df.columns

Index(['Outcome', 'Text', 'Id'], dtype='object')

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

#test sid
sid.polarity_scores(df.Text[0])['compound']


0.3182

In [13]:
df = df \
    .assign(vader_sentiment = df.Text[:100000].progress_apply(lambda text: sid.polarity_scores(text)['compound']))

100%|██████████| 100000/100000 [00:37<00:00, 2678.58it/s]


In [11]:
import swifter

df['vader_sentiment_2'] = df.Text.swifter.apply(lambda text: sid.polarity_scores(text)['compound'])

Pandas Apply: 100%|██████████| 955454/955454 [06:00<00:00, 2646.77it/s]


In [14]:
df.head(5)

,Outcome,Text,Id,vader_sentiment,vader_sentiment_2
0,14,I am having a problem with the first example o...,1,0.3182,0.3182
1,14,"everyone, I met a tough definite integral as f...",2,0.1531,0.1531
2,7,"Please dont lynch me, but i've never sat throu...",3,-0.7789,-0.7789
3,14,How to calculate $ \mathbb{Z}[x] /\langle2x-1\...,4,0.0000,0.0000
4,2,When somebody rings or texts my iPhone it is n...,5,0.0000,0.0000


In [4]:
import spacy
from bs4 import BeautifulSoup
import re

nlp = spacy.load('en_core_web_sm')

In [5]:
def preprocess_soup(text):
    bs = BeautifulSoup(text, 'lxml').get_text()
    return bs

In [ ]:
df['Preprocess'] = df.Text.progress_apply(preprocess_soup)

In [11]:
from nltk.corpus import stopwords
import re
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer 

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocess(input):
    # removing link
    pat1 = r'http?://[A-Za-z0-9./]+'
    pat2 = r'https?://[A-Za-z0-9./]+'
    combined_pat = r'|'.join((pat1, pat2))
    re1 = re.sub(combined_pat, '', input)

    # lower case
    text = re1.lower()
    
    #remove apostrophy
    remove_apo =  text.translate({ord(c): "" for c in "'"})
    
    #remove special character
    removeSpecialChars =  remove_apo.translate({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
    words = removeSpecialChars.split()

    # loop for preprocessing
    sentence_transform = []
    for word in words:
        word_transform = ""
        for character in word:
            if character.isalpha():
                word_transform += character
        if (word_transform not in stop_words) & (len(word_transform) > 1):
            if not(word_transform.isdigit()):
                word_lemmatize = lemmatizer.lemmatize(word_transform)
                sentence_transform.append(word_lemmatize)
    
    if len(sentence_transform) >= 100:
        output = ' '.join(sentence_transform[:100])
    else:
        output = ' '.join(sentence_transform)
    
    output = re.sub(' +', ' ', output)
    
    return output

In [14]:
preprocess(df.Preprocess[0])

'problem first example amir dembo ofer zeitouni book large deviation technique application could someone please help confirm following statement standard normal random variable logp sum ge delta delta infty'

In [16]:
df['Output'] = df.Preprocess.progress_apply(preprocess)

100%|██████████| 955454/955454 [03:30<00:00, 4544.26it/s]


In [20]:
df \
    [['Id', 'Output', 'Outcome']] \
    .to_csv('../data/preprocess/train_prep.csv')